In [ ]:
# Installs for the below cells.

# !pip install openai
# !pip install langchain
# !pip install tiktoken
# !pip install faiss-cpu
# !pip install beautifulsoup4
# !pip install -U duckduckgo-search==3.9.10

# Introduction to LangChain Expression Language ([LCEL](https://python.langchain.com/docs/get_started/introduction))

LCEL is a declarative way to compose chains of components. 

What does that mean? Means its an easy way to put useful building blocks together.


Here's quick summary of the LangChain Expression Language (LCEL) page:

- LCEL Basics: Simplifies building complex chains from basic components using a unified interface and composition primitives.

- Unified Interface: Every LCEL object implements the Runnable interface, supporting common invocation methods like invoke, batch, stream, ainvoke, and more.

- Composition Primitives: LCEL provides tools for composing chains, parallelizing components, adding fallbacks, and dynamically configuring internal chain elements.

- Model Flexibility: LCEL allows for easy switching between different models and providers (like OpenAI or Anthropic), and runtime configurability of chat models or LLMs.

- Advanced Features: LCEL features things like logging intermediate results with LangSmith integration and adding fallback logic for enhanced reliability.

Ok, cool but what is a component?

A component is something that implements the `Runnable` protocol.


Ok....and what is that?

It's an object with some nice desirable features like:

- input and output schemas (describe what that object takes as input and the structure of its output)

Some nice methods are:

- `invoke` [ainvoke]
- `stream` [astream]
- `batch` [abatch]


Below is a list of common i/o types for each component:

![](./assets-resources/components_input_type_output_type.png)

[source for the image](https://python.langchain.com/docs/expression_language/interface)

Let's look at a simple example.

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm_chat = ChatOpenAI()
prompt = ChatPromptTemplate.from_template(("List 5 examples of applications for this concept: {concept}"))
output_parser = StrOutputParser()

chain = prompt | llm_chat | output_parser

chain.invoke({"concept": "machine learning"})

Ok nice! So we put everything together using this [pipe](https://en.wikipedia.org/wiki/Pipeline_(Unix)) `|` symbol (or [unix pipe operator](https://en.wikipedia.org/wiki/Pipeline_(Unix)) if you want to get fancy) That's the power of the LCEL language, putting different components together through a simple interface.

[source](https://python.langchain.com/docs/expression_language/get_started#:~:text=its%20cone-fidence!%22-,4.%20entire%20pipeline)

So what is happening is:

- We pass in user input on the desired concept as {"concept": "machine learning"}
- The prompt component takes the user input, which is then used to construct a `PromptValue` after using the `concept` to construct the `prompt`.
- The model component takes the generated prompt, and passes into the OpenAI LLM model for evaluation. The generated output from the model is a `ChatMessage` object.
- Finally, the `output_parser` component takes in a `ChatMessage`, and transforms this into a Python string, which is returned from the invoke method.

```mermaid
graph LR;
    A[Input concept=machine learning] --dict--> B[Prompt Template]
    B --PromptValue--> C[ChatModel]
    C --ChatMessage--> D[StrOutputParser]
    D --string--> E[Output]
```

Here's a bullet point summary of the key features and benefits of LangChain Expression Language (LCEL):

Declarative Composing: LCEL allows for easy composition of chains, ranging from simple "prompt + LLM" chains to complex ones with hundreds of steps.

Streaming Support: LCEL offers optimal time-to-first-token, enabling streaming of tokens from an LLM to a streaming output parser for quick, incremental output.

Async Support: Chains built with LCEL can be used both synchronously (e.g., in Jupyter notebooks for prototyping) and asynchronously (e.g., in a LangServe server), maintaining consistent code for prototypes and production.

Optimized Parallel Execution: LCEL automatically executes parallel steps in a chain (like fetching documents from multiple retrievers) in both sync and async interfaces, reducing latency.

Retries and Fallbacks: Users can configure retries and fallbacks for any part of the LCEL chain, enhancing reliability at scale. Streaming support for these features is in development.

Access to Intermediate Results: LCEL allows access to intermediate step results, useful for user updates or debugging. This feature includes streaming intermediate results and is available on all LangServe servers.

Input and Output Schemas: LCEL chains come with Pydantic and JSONSchema schemas, inferred from the chain's structure, which aid in validating inputs and outputs. This is a core part of LangServe.

Seamless LangSmith Tracing Integration: As chains become more complex, LCEL provides automatic logging of all steps to LangSmith for enhanced observability and debuggability.

Seamless LangServe Deployment Integration: LCEL chains can be easily deployed using LangServe, facilitating smoother deployment processes.

These features highlight LCEL's versatility and efficiency in both development and production environments, making it a powerful tool for creating and managing complex language chains.

Now, let's look into the `Runnable` components in more detail. 

`RunnableLambda`

In [ ]:
from langchain.schema.runnable import RunnableLambda

# Define a custom function
def add_one(x: int) -> int:
    return x + 1

# Create a RunnableLambda and invoke it
runnable = RunnableLambda(add_one)
output = runnable.invoke(5)
print(output)  # Output: 6

[`RunnableSequence`](https://api.python.langchain.com/en/stable/runnables/langchain_core.runnables.base.Runnable.html#langchain_core.runnables.base.Runnable:~:text=runnablesequence%20invokes%20a%20series%20of%20runnables%20sequentially%2C%20with%20one%20runnable%E2%80%99s%20output%20serving%20as%20the%20next%E2%80%99s%20input.%20construct%20using%20the%20%7C%20operator%20or%20by%20passing%20a%20list%20of%20runnables%20to%20runnablesequence.)

RunnableSequence invokes a series of runnables sequentially, with one runnable’s output serving as the next’s input. Construct using the | operator or by passing a list of runnables to RunnableSequence.

In [ ]:
from langchain_core.runnables import RunnableSequence
# Suppose we have a list of numbers
data = [1, 2, 3, 4, 5]

# Preprocess function: Multiply each number by 2
def preprocess(data):
    return [x * 2 for x in data]

# Apply model function: Square each number
def apply_model(data):
    return [x ** 2 for x in data]

# Postprocess function: Sum all the numbers
def postprocess(data):
    return sum(data)

# Create RunnableLambdas
preprocess_runnable = RunnableLambda(preprocess)
apply_model_runnable = RunnableLambda(apply_model)
postprocess_runnable = RunnableLambda(postprocess)

# Create a RunnableSequence and invoke it
sequence = preprocess_runnable | apply_model_runnable | postprocess_runnable
# runnable_sequence = RunnableSequence(sequence)
output = sequence.invoke(data)

print(output)
# or
runnable_sequence = RunnableSequence(first=preprocess_runnable, middle=[apply_model_runnable], last=postprocess_runnable)
runnable_sequence.invoke(data)

[`Runnableparallel`]([`Runnableparallel`](https://api.python.langchain.com/en/stable/runnables/langchain_core.runnables.base.Runnable.html#langchain_core.runnables.base.Runnable:~:text=runnableparallel%20invokes%20runnables%20concurrently%2C%20providing%20the%20same%20input%20to%20each.%20construct%20it%20using%20a%20dict%20literal%20within%20a%20sequence%20or%20by%20passing%20a%20dict%20to%20runnableparallel.)https://api.python.langchain.com/en/stable/runnables/langchain_core.runnables.base.Runnable.html#langchain_core.runnables.base.Runnable:~:text=runnableparallel%20invokes%20runnables%20concurrently%2C%20providing%20the%20same%20input%20to%20each.%20construct%20it%20using%20a%20dict%20literal%20within%20a%20sequence%20or%20by%20passing%20a%20dict%20to%20runnableparallel.)

RunnableParallel invokes runnables concurrently, providing the same input to each. Construct it using a dict literal within a sequence or by passing a dict to RunnableParallel.

In [ ]:
import langchain
print(langchain.__version__)

In [ ]:
from langchain.schema.runnable import RunnableLambda

# A RunnableSequence constructed using the `|` operator
sequence = RunnableLambda(lambda x: x + 1) | RunnableLambda(lambda x: x * 2)
sequence.invoke(1) # 4
sequence.batch([1, 2, 3]) # [4, 6, 8]

# A sequence that contains a RunnableParallel constructed using a dict literal
sequence = RunnableLambda(lambda x: x + 1) | {
    'mul_2': RunnableLambda(lambda x: x * 2),
    'mul_5': RunnableLambda(lambda x: x * 5)
}
sequence.invoke(1) # {'mul_2': 4, 'mul_5': 10}


When you build chains, what you're doing is building a type of Runnable!

That could be a RunnableSequence or a RunnableParallel  

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI


llm = ChatOpenAI()
prompt = PromptTemplate.from_template("Summarize this {text}")
prompt2 = PromptTemplate.from_template("Explain this {text}")


chain1 = prompt | llm

chain2 = prompt2 | llm

runnable_parallel = chain1 

In [ ]:
type(chain)

[RunnableParallel can be useful for manipulating the output of one Runnable to match the input format of the next Runnable in a sequence.](https://python.langchain.com/docs/expression_language/how_to/map#:~:text=RunnableParallel%20can%20be,the%0A%E2%80%9Cquestion%E2%80%9D%20key.)

RunnableParallel (aka. RunnableMap) makes it easy to execute multiple Runnables in parallel, and to return the output of these Runnables as a map.

([see it in docs here](https://python.langchain.com/docs/expression_language/how_to/map#:~:text=runnableparallel%20(aka.%20runnablemap)%20makes%20it%20easy%20to%20execute%20multiple%20runnables%20in%20parallel%2C%20and%20to%20return%20the%20output%20of%20these%20runnables%20as%20a%20map.))

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnableParallel

model = ChatOpenAI()
joke_chain = ChatPromptTemplate.from_template("tell me a joke about {topic}") | model
poem_chain = (
    ChatPromptTemplate.from_template("write a 2-line poem about {topic}") | model
)

map_chain = RunnableParallel(joke=joke_chain, poem=poem_chain)

map_chain.invoke({"topic": "bear"})

RunnableParallel are also useful for running independent processes in parallel, since each Runnable in the map is executed in parallel. For example, we can see our earlier joke_chain, poem_chain and map_chain all have about the same runtime, even though map_chain executes both of the other two.

In [ ]:
%%timeit

joke_chain.invoke({"topic": "bear"})

In [ ]:
%%timeit

poem_chain.invoke({"topic": "bear"})

In [ ]:
%%timeit

map_chain.invoke({"topic": "bear"})

[RunnablePassthrough](https://python.langchain.com/docs/expression_language/how_to/passthrough#:~:text=RunnablePassthrough%20allows%20to,pass%20it%20through.)

Allows you to pass inputs unchanged or with addition of new keys.

Usually you would connect this with `RunnableParallel` to assign data to new key in the map.

If you just call it, it will take the input and pass it along.

In [ ]:
from langchain.schema.runnable import RunnablePassthrough

# Create a RunnablePassthrough
passthrough = RunnablePassthrough()
passthrough.invoke(5) # 5

But if you call it with the `.assign(...)` mthod, then it will take the input, and add extra arguments passed to the assign function. 

The key being added has to be a lambda function.

In [ ]:
from langchain.schema.runnable import RunnableParallel, RunnablePassthrough

# Passthrough with assignment, adds an extra key-value pair
runnable = RunnablePassthrough.assign(extra_key_add_5=lambda x:  x["num"]+5)
input_data = {"num": 1}
result = runnable.invoke(input_data)
# Output: {'num': 1, 'extra_value': 42}
result

In combination with RunnableParallel:

In [ ]:
runnable = RunnableParallel(
    origin=RunnablePassthrough(),
    modified=lambda x: x+1
)

runnable.invoke(1) # {'origin': 1, 'modified': 2}

In [ ]:
# modified example from langchain docs: https://api.python.langchain.com/en/stable/runnables/langchain_core.runnables.passthrough.RunnablePassthrough.html?highlight=runnablepassthrough#langchain_core.runnables.passthrough.RunnablePassthrough 
# or for an "LLM" example:

def fake_llm(prompt: str) -> str: # Fake LLM for the example
    return "completion"

chain = RunnableLambda(fake_llm) | {
    'original': RunnablePassthrough(), # Original LLM output
    'parsed': lambda text: text[::-1] + " (applied the parsing logic)" # Parsing logic
}

chain.invoke('hello') # {'original': 'completion', 'parsed': 'noitelpmoc'}

Ok so we have these 4 types of main objects:

- `RunnableLambda`
- `RunnableSequence`
- `RunnablePassthrough`
- `RunnableParallel`

combined they make up kind of the core building block system of langchain allow you to create complex inner logics powered by llms. 

Let's create some fun examples below:

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough

def fun_llm(prompt: str):
    return ChatOpenAI().predict(f"Make this funny: {prompt}")


def corporate_llm(prompt: str):
    return ChatOpenAI().predict(f"Make this topic a subject of a very corporate email: {prompt}")

fun_chain = RunnableLambda(fun_llm) | {
    "original-text-input": RunnablePassthrough(), 
    "make-it-corporate": lambda x: corporate_llm(x)
}


fun_chain.invoke("gorillas")

In [ ]:
from langchain.schema.runnable import RunnablePassthrough, RunnableParallel

def fake_llm(prompt: str) -> str: # Fake LLM for the example
    return "completion"

runnable = {
    'llm1':  fake_llm,
    'llm2':  fake_llm,
} | RunnablePassthrough.assign(
    total_chars=lambda inputs: len(inputs['llm1'] + inputs['llm2'])
  )

runnable.invoke('hello')
# {'llm1': 'completion', 'llm2': 'completion', 'total_chars': 20}

Breaking down whats happening:

We create a `runnable` object, which is an instance of the `RunableSequence` object. This chain takes in as input a simple string: `hello`, and what it does is it 
simulates passing that same string to 2 `different` llms (which in this case are the same) and applies the logic of each of these llms as well as a third logic described by the new extra key added by the RunnablePassthrough.assign() method, that takes as input the output of the `llm1` and `llm2` as input to combine it through the lambda function associated with the new extra key created: `total_chars`. 

Let's make this example a bit more interesting by applying some research workflow vibe to it:

In [ ]:
# let's create a couple of llms to do different things with some piece of content.
# In this case let's use different llms to create summarization levels for some text.
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

def llm_summarization_level1(prompt: str):
    return ChatOpenAI().predict(f"Summarize this text: {prompt}")

def llm_summarization_level2(prompt: str):
    return ChatOpenAI().predict(f"Summarize this text in bullet points: {prompt}")

def llm_summarization_level3(prompt: str):
    return ChatOpenAI().predict(f"Summarize this text into one short paragraph: {prompt}")


# now let's create a chain that will run all of these llms in parallel and then return the results

runnable = {
    'level1':  llm_summarization_level1,
    'level2':  llm_summarization_level2,
    'level3':  llm_summarization_level3,
} | RunnablePassthrough.assign(prompt_level4=lambda x: f"Combine these 3 summarizations\
    into one combining all their \n \
    useful features. \
    Level1: {x['level1']}.\n \
    Level2: {x['level2']}.\n\
    Level3: {x['level3']}"
                       ) | RunnablePassthrough.assign(output=RunnableLambda(lambda x: ChatOpenAI().predict(x['prompt_level4']))) 
# For the assign method with RunningPasshthrough, we'll combine the summarizations into a fourth one.

runnable.invoke(
    """
    The code you provided is from the nest_asyncio library, which is designed to patch Python's asyncio library to allow nested usage of the asyncio event loop. To understand this code, it's crucial first to grasp what an event loop is in the context of asynchronous programming.
What is an Event Loop?
An event loop in programming, particularly in asynchronous programming, is a central control structure that manages and dispatches events or messages in a program. In the context of Python's asyncio library, the event loop is a core feature that runs asynchronous tasks and callbacks, handles network IO operations, and manages subprocesses. It is essentially the heart of the asyncio module, enabling asynchronous programming by juggling and scheduling the execution of various tasks.
Key Concepts in the nest_asyncio Code
Patching asyncio: The code modifies (patches) certain parts of the asyncio library. This is done to change the default behavior of asyncio, particularly to support nested event loops, which are not allowed in standard asyncio.
Reentrancy in Event Loops: The primary function of nest_asyncio is to make the asyncio event loop reentrant. In computing, reentrancy refers to the ability of a function to be paused in the middle of execution and safely called again ("re-entered") before its previous executions are complete. This is not normally supported by the asyncio event loops, as they are designed to prevent re-entry (or nesting) to avoid complex problems and unexpected behavior.
Modifying Loop Behavior: The code alters the behavior of the event loop methods like run_forever, run_until_complete, and the internal _run_once. These modifications allow the event loop to pause and resume (re-enter) gracefully, facilitating nesting.
Context Managers: The code uses context managers (manage_run and manage_asyncgens) to properly manage the state of the event loop during entry and exit of asynchronous contexts, which is crucial for handling nested loops correctly.
Task Patching: It also patches the Task class of asyncio to modify its step function. This is necessary to ensure that the tasks (units of work scheduled by the event loop) behave correctly in a nested loop scenario.
Tornado Patching: If the Tornado library (an asynchronous networking library) is used, nest_asyncio makes Tornado aware of the Python asyncio Future, ensuring compatibility.
Understanding the Event Loop in Async Programming
In asynchronous programming, especially in Python's asyncio, the event loop is pivotal. It allows the execution of multiple tasks seemingly in parallel by switching between them. This switching is non-blocking, meaning the program can handle other tasks while waiting for some IO operation to complete, thereby increasing efficiency and responsiveness.

The nest_asyncio library's primary role is to tweak the asyncio's event loop to support nested operation, which is particularly useful in scenarios like running an asyncio event loop inside another asyncio event loop, something that standard asyncio does not support by default.

For more details on asynchronous programming and event loops in Python, the Python asyncio documentation is a comprehensive resource.
"""
)

A simple research assistant, heavily based on this implementation by Harrison Chase:
- https://gist.github.com/hwchase17/69a8cdef9b01760c244324339ab64f0c

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
import requests
from bs4 import BeautifulSoup
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.utilities import DuckDuckGoSearchAPIWrapper
import json

RESULTS_PER_QUESTION = 1

ddg_search = DuckDuckGoSearchAPIWrapper()


def web_search(query: str, num_results: int = RESULTS_PER_QUESTION):
    results = ddg_search.results(query, num_results)
    return [r["link"] for r in results]


def scrape_text(url: str):
    # Send a GET request to the webpage
    try:
        response = requests.get(url)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the content of the request with BeautifulSoup
            soup = BeautifulSoup(response.text, "html.parser")

            # Extract all text from the webpage
            page_text = soup.get_text(separator=" ", strip=True)

            # Print the extracted text
            return page_text
        else:
            return f"Failed to retrieve the webpage: Status code {response.status_code}"
    except Exception as e:
        print(e)
        return f"Failed to retrieve the webpage: {e}"


def collapse_list_of_lists(list_of_lists):
    content = []
    for l in list_of_lists:
        content.append("\n\n".join(l))
    return "\n\n".join(content)

In [ ]:
SUMMARY_TEMPLATE = """{text} 
-----------
Using the above text, answer in short the following question: 
> {question}
-----------
if the question cannot be answered using the text, imply summarize the text. Include all factual information, numbers, stats etc if available."""  # noqa: E501
SUMMARY_PROMPT = ChatPromptTemplate.from_template(SUMMARY_TEMPLATE)


url = "https://blog.langchain.dev/announcing-langsmith/"

scrape_and_summarize_chain = RunnablePassthrough.assign(
    summary = RunnablePassthrough.assign(
    text=lambda x: scrape_text(x["url"])[:10000]
) | SUMMARY_PROMPT | ChatOpenAI(model="gpt-3.5-turbo-1106") | StrOutputParser()
) | (lambda x: f"URL: {x['url']}\n\nSUMMARY: {x['summary']}")

# Let's see the input_schema of our created chain
print(scrape_and_summarize_chain.input_schema())

At this stage, even though this chain requires two keys in the input dictionary, that's not reflected in the input_schema() of the chain, because in it only goes the variables contained in the ChatPromptTemplate(). 

Interestingly enough, the `text` variable which is in the ChatPromptTemplate, is not in the schema because its already part of the chain itself (therefore not being necessary in the input_schema() I guess).

In [ ]:
# At this
# let's test this chain on some url
scrape_and_summarize_chain.invoke({"question": "What is langsmith?", "url": url})

In [ ]:
web_search_chain = RunnablePassthrough.assign(
    urls = lambda x: web_search(x["question"]) # urls will be the output of the web_search function()
) | (lambda x: [{"question": x["question"], "url": u} for u in x["urls"]]) | scrape_and_summarize_chain.map()

Let's now test this web search chain.

In [ ]:
web_search_chain.invoke({"question": "Look up from langchain.chains import example_generator"})

In [ ]:
SEARCH_PROMPT = ChatPromptTemplate.from_messages(
    [
        (
            "user",
            "Write 3 google search queries to search online that form an "
            "objective opinion from the following: {question}\n"
            "You must respond with a list of strings in the following format: "
            '["query 1", "query 2", "query 3"].',
        ),
    ]
)

search_question_chain = SEARCH_PROMPT | ChatOpenAI(temperature=0) | StrOutputParser() | json.loads

In [ ]:
search_question_chain.invoke({"question": "What are the most useful langchain utilities for research?"})

In [ ]:
full_research_chain = search_question_chain | (lambda x: [{"question": q} for q in x]) | web_search_chain.map()

In [ ]:
full_research_chain.invoke({"question": "What is langchain?"})

In [ ]:
WRITER_SYSTEM_PROMPT = "You are an AI critical thinker research assistant. Your sole purpose is to write well written, critically acclaimed, objective and structured reports on given text."  # noqa: E501


# Report prompts from https://github.com/assafelovic/gpt-researcher/blob/master/gpt_researcher/master/prompts.py
RESEARCH_REPORT_TEMPLATE = """Information:
--------
{research_summary}
--------
Using the above information, answer the following question or topic: "{question}" in a detailed report -- \
The report should focus on the answer to the question, should be well structured, informative, \
in depth, with facts and numbers if available and a minimum of 1,200 words.
You should strive to write the report as long as you can using all relevant and necessary information provided.
You must write the report with markdown syntax.
You MUST determine your own concrete and valid opinion based on the given information. Do NOT deter to general and meaningless conclusions.
Write all used source urls at the end of the report, and make sure to not add duplicated sources, but only one reference for each.
You must write the report in apa format.
Please do your best, this is very important to my career."""  # noqa: E501

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", WRITER_SYSTEM_PROMPT),
        ("user", RESEARCH_REPORT_TEMPLATE),
    ]
)


chain = RunnablePassthrough.assign(
    research_summary= full_research_chain | collapse_list_of_lists
) | prompt | ChatOpenAI(model="gpt-3.5-turbo-1106") | StrOutputParser()


`RunnableMap`

> Let's modify this example for something that uses the updated version of the pydantic library.

In [37]:

# !pip install pydantic==1.10.8

The folder you are executing pip from can no longer be found.


In [2]:
# this example requires pydantic==
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnableMap
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser


llm_chat = ChatOpenAI()

vectorstore = DocArrayInMemorySearch.from_texts(
    ["harrison worked at kensho", "bears like to eat honey"],
    embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

retriever.get_relevant_documents("Where did harrison work?")

template = "Answer the question based on this context: {context}\n\nQuestion: {question}"
prompt = ChatPromptTemplate.from_template(template)

output_parser = StrOutputParser()

chain = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"],
    
}) | prompt | llm_chat | output_parser



chain.invoke({"question": "What did harrison do?"})

'Harrison worked at Kensho.'

If we inspect just the `RunnableMap`:

In [6]:
from langchain.schema.runnable import RunnablePassthrough
inputs_with_lambda = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"],
})

inputs_with_lambda.invoke({"question": "What did harrison do?"})

{'context': [Document(page_content='harrison worked at kensho'),
  Document(page_content='bears like to eat honey')],
 'question': 'What did harrison do?'}

In [7]:
inputs_with_runnable_passthrough = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"],
})

inputs_with_runnable_passthrough.invoke({"question": "What did harrison do?"})

{'context': [Document(page_content='harrison worked at kensho'),
  Document(page_content='bears like to eat honey')],
 'question': 'What did harrison do?'}

You get the same thing!

You can also use tools with runnables in some neat and interesting ways. Let's see that in the next notebook (2.2)

We see that what the `RunnableMap` is doing, is create the right inputs for the downstream of this chain which in this case is seeting the `context` key with a value of a list of `Document` objects, and the question key with itself.
(which now we can probably change to something simpler with the `RunnablePassthrough` for example).